In [ ]:
from helper import*
from params import*

In [ ]:
# establish connection with music_therapy_eeg database
db_connection = create_db_connection(hostName,userName,userPassword,databaseName)
data = df_to_table_query(db_connection,query)

In [ ]:
# transform sql extracted table to raw eeg data
rawEEG = (transformToRawEEG(data,fs,collection_time,fs_setting))[0]
time_s = (transformToRawEEG(data,fs,collection_time,fs_setting))[1]
plots(time_s,rawEEG,plotTitles,figSize,pltColor)

In [ ]:
# compute independent comonent analysis on raw eeg data
ica_data = customICA(rawEEG,tuneval)
plots(time_s,ica_data,plotTitles,figSize,pltColor)

In [ ]:
# Filters: notch filter and bandpass filter
noc=filters()
notch_data = noc.notch(ica_data,line,fs,Q)
bp = filters()
bpData = bp.butterBandPass(notch_data,lowcut,highcut,fs,order)
plots(time_s,bpData,plotTitles,figSize,pltColor)